In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pydicom
import random

In [3]:
ROOT = CONFIG.CFG.DATA.BASE
BATCH_SIZE = 64

In [4]:
train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [83]:
class OSICLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(OSICLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        out, _ = self.lstm(X)
        out = self.lc(out)
        return out

In [88]:
model = OSICLSTM(4, 100, 1, 4)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [76]:
len(list(model.parameters()))

6

In [35]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])


In [77]:
print(model)

OSICLSTM(
  (lstm): LSTM(4, 100, batch_first=True)
  (lc): Linear(in_features=100, out_features=4, bias=True)
)


In [38]:
ALL_PATIENTS = train_df["Patient"].unique().tolist()

In [85]:
model.train()

OSICLSTM(
  (lstm): LSTM(4, 100, batch_first=True)
  (lc): Linear(in_features=100, out_features=4, bias=True)
)

In [89]:
for patient in ALL_PATIENTS:
    patient_data = train_df[train_df["Patient"] == patient][['Weeks', 'FVC', 'Percent', 'Age']].values
    patient_data = torch.tensor(patient_data).float()

    for data in patient_data:
        out = model(data.view(1,1,-1))
        print(out)
        break
    break

tensor([[[-0.0150, -0.4470,  0.1776,  0.2869]]], grad_fn=<AddBackward0>)


In [87]:
for epoch in range(1000):
    total_loss = 0
    for patient in ALL_PATIENTS:
        model.zero_grad()

        patient_data = train_df[train_df["Patient"] == patient][['Weeks', 'FVC', 'Percent', 'Age']].values
        patient_data = torch.tensor(patient_data).float()

        loss = 0

        for i in range(patient_data.size()[0] - 1):
            out = model(data.view(1,1,-1))
            loss += loss_function(out, patient_data[i+1])
            total_loss += loss.item()
        # for data in patient_data:
        #     out = model(data.view(1, 1, -1))
        #     print(out)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, loss={total_loss}")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([1, 1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 0, loss=1465597586.6549072
Epoch 1, loss=1381502054.5233154
Epoch 2, loss=1381502023.5220337
Epoch 3, loss=1381502024.964386
Epoch 4, loss=1381502024.553543
Epoch 5, loss=1381502024.5938416
Epoch 6, loss=1381502024.564148
Epoch 7, loss=1381502024.905304
Epoch 8, loss=1381502024.6339417
Epoch 9, loss=1381502024.8549957
Epoch 10, loss=1381502024.718628
Epoch 11, loss=1381502024.9852295
Epoch 12, loss=1381502024.670227
Epoch 13, loss=1381502024.566986
Epoch 14, loss=1381502024.84729
Epoch 15, loss=1381502024.7083282
Epoch 16, loss=1381502024.8101807
Epoch 17, loss=1381502024.5385742
Epoch 18, loss=1381502024.5575867
Epoch 19, loss=1381502024.53

KeyboardInterrupt: 

In [27]:
with torch.no_grad():
    input = torch.randn(2, 5, 4)
    print(input)
    print()
    out = model(input)
    print(out)
    print(out.shape)

tensor([[[-0.5860,  1.3153, -1.0502, -0.5227],
         [-1.1907, -1.2158, -0.4712, -0.1658],
         [-0.9553,  0.9396, -1.2907,  0.4846],
         [-0.1314, -0.4851,  0.1461,  0.1841],
         [ 0.1598, -0.7560,  0.1895,  0.5957]],

        [[-0.9853,  0.9842, -0.3083, -0.5643],
         [ 1.4160, -0.1827,  0.5166, -0.1190],
         [-0.0798, -0.3892, -0.3054,  0.2493],
         [-0.3142, -2.8067, -1.8554, -0.9012],
         [ 0.9378, -0.8356,  2.2742,  1.2010]]])

tensor([[[ 0.1448, -0.0468, -0.0903, -0.0212,  0.0090, -0.0891, -0.0880,
          -0.0046, -0.0033, -0.0360],
         [ 0.2230, -0.0832, -0.1429, -0.0101,  0.0148, -0.1451, -0.1288,
           0.0017,  0.0270, -0.0820],
         [ 0.2773, -0.0919, -0.1637, -0.0215,  0.0170, -0.1744, -0.1577,
          -0.0025,  0.0194, -0.1195],
         [ 0.2993, -0.0928, -0.1835, -0.0177,  0.0165, -0.1946, -0.1785,
          -0.0057,  0.0355, -0.1458],
         [ 0.3126, -0.0837, -0.1999, -0.0143,  0.0147, -0.2093, -0.1937,
        